In [1]:
#S=> stand for single responsibility 
# Bad example 

class Report:
    def generate_report(self):
        # logic to generate report
        pass

    def save_to_file(self, filename):
        # logic to save report to a file
        pass


#why this is a bad example?
#class Report generates a report and saves it to files if any error happened in saving the whole class crashes 



#good example to fix that 

class Report:
    def generate_report(self):
        # logic to generate report
        pass

class ReportSaver:
    def save_to_file(self, report, filename):
        # logic to save report to a file
        pass


# usually we use compostition when applying this principle   


In [2]:
#O⇒ stands for Open-Closed Open for exytension , closed for modification
# bad example
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

# AreaCalculator only works with Rectangle (not extensible)
class AreaCalculator:
    def calculate_area(self, rectangle):
        return rectangle.area()

# Problem: If we add a Circle, we must modify AreaCalculator!
#why this is a bad example ? because what if i need to add another shape other than rectangle ?


# good example to fix that 
from abc import ABC, abstractmethod

# Abstraction (Open for extension)
class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

# Concrete implementations (can be extended without modifying existing code)
class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14 * self.radius ** 2

# AreaCalculator now works with ANY Shape (Closed for modification)
class AreaCalculator:
    def calculate_area(self, shape: Shape):
        return shape.area()

# Usage
rectangle = Rectangle(10, 5)
circle = Circle(7)

calculator = AreaCalculator()
print(calculator.calculate_area(rectangle))  # Output: 50
print(calculator.calculate_area(circle))     # Output: 153.86

# Now, adding a new shape (e.g., Triangle) doesn't require changing AreaCalculator!    

50
153.86


In [3]:
#L⇒ stands for Liskov Substitution
#If a class B is a subclass of A, then you should be able to pass B wherever A is expected without causing errors or unexpected behavior.
# bad examnple on that 
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def set_width(self, width):
        self.width = width

    def set_height(self, height):
        self.height = height

    def area(self):
        return self.width * self.height

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)

    def set_width(self, width):
        self.width = width
        self.height = width  # Forces height to equal width (violates Rectangle's behavior)

    def set_height(self, height):
        self.height = height
        self.width = height  # Forces width to equal height (violates Rectangle's behavior)

# Client code expects Rectangle behavior
def test_rectangle(rectangle):
    rectangle.set_width(5)
    rectangle.set_height(4)
    expected_area = 20
    assert rectangle.area() == expected_area, f"Expected area: {expected_area}, got: {rectangle.area()}"

# Works fine with Rectangle
rect = Rectangle(5, 4)
test_rectangle(rect)  # ✅ Passes

# Breaks with Square (violates LSP)
square = Square(5)
test_rectangle(square)  # ❌ Fails (area becomes 16, not 20) 
#why this is a bad example?
#beacuse the child class can't operate the sane as the parent and somehow the logic of the program crashes 


# good example
from abc import ABC, abstractmethod

# Abstraction (common interface)
class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

# Rectangle and Square are siblings, not parent-child
class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

class Square(Shape):
    def __init__(self, side):
        self.side = side

    def area(self):
        return self.side ** 2

# Client code works with Shape (no assumptions about setters)
def print_area(shape: Shape):
    print(f"Area: {shape.area()}")

# Both work correctly without violating LSP
rectangle = Rectangle(5, 4)
square = Square(5)

print_area(rectangle)  # Output: Area: 20
print_area(square)     # Output: Area: 25    # Output: Area: 25

AssertionError: Expected area: 20, got: 16

In [5]:
#I⇒ Interface Segregation
#it's better to have many small, specific interfaces than one large, general-purpose interface.

#Bad Example
from abc import ABC, abstractmethod

class Machine(ABC):
    @abstractmethod
    def print(self, document):
        pass
    
    @abstractmethod
    def fax(self, document):
        pass
    
    @abstractmethod
    def scan(self, document):
        pass

# Old-fashioned printer can only print
class OldPrinter(Machine):
    def print(self, document):
        print(f"Printing {document}")
    
    def fax(self, document):
        raise NotImplementedError("Fax not supported")
    
    def scan(self, document):
        raise NotImplementedError("Scan not supported")

# Modern printer can do everything
class ModernPrinter(Machine):
    def print(self, document):
        print(f"Printing {document}")
    
    def fax(self, document):
        print(f"Faxing {document}")
    
    def scan(self, document):
        print(f"Scanning {document}")
#why this is a bad example?
#because the old printer has to do things it can't implement so we need to split the interfaces rather than having one bigh interface that makes 
#no sense
  
  
  #Good Example to fix that 

from abc import ABC, abstractmethod

class Printer(ABC):
    @abstractmethod
    def print(self, document):
        pass

class Fax(ABC):
    @abstractmethod
    def fax(self, document):
        pass

class Scanner(ABC):
    @abstractmethod
    def scan(self, document):
        pass

class OldPrinter(Printer):
    def print(self, document):
        print(f"Printing {document}")

class Photocopier(Printer, Scanner):
    def print(self, document):
        print(f"Printing {document}")
    
    def scan(self, document):
        print(f"Scanning {document}")

class MultiFunctionDevice(Printer, Fax, Scanner):
    def print(self, document):
        print(f"Printing {document}")
    
    def fax(self, document):
        print(f"Faxing {document}")
    
    def scan(self, document):
        print(f"Scanning {document}") #Dependency Inversion  
    


In [6]:
#D⇒  Dependency Inversion
#High-Level Module → Abstraction ← Low-Level Module  which means
#High-level modules should not depend on low-level modules. Both should depend on abstractions.
#bad example

class MySQLDatabase:
    def insert(self, data):
        print(f"Inserting {data} into MySQL database")
    
    def update(self, id, data):
        print(f"Updating {id} with {data} in MySQL database")

class Application:
    def __init__(self):
        self.db = MySQLDatabase()  # Direct dependency
    
    def save_data(self, data):
        self.db.insert(data)
    
    def update_data(self, id, data):
        self.db.update(id, data)

#why this is a bad example?
#because the high level class depends directly on the low level class

# Good Example to fix that


from abc import ABC, abstractmethod

class Database(ABC):
    @abstractmethod
    def insert(self, data):
        pass
    
    @abstractmethod
    def update(self, id, data):
        pass

class MySQLDatabase(Database):
    def insert(self, data):
        print(f"Inserting {data} into MySQL database")
    
    def update(self, id, data):
        print(f"Updating {id} with {data} in MySQL database")

class PostgreSQLDatabase(Database):
    def insert(self, data):
        print(f"Inserting {data} into PostgreSQL database")
    
    def update(self, id, data):
        print(f"Updating {id} with {data} in PostgreSQL database")

class Application:
    def __init__(self, database: Database):  # Depends on abstraction
        self.db = database
    
    def save_data(self, data):
        self.db.insert(data)
    
    def update_data(self, id, data):
        self.db.update(id, data)

# Usage
mysql_db = MySQLDatabase()
app_with_mysql = Application(mysql_db)
app_with_mysql.save_data({"name": "John"})

postgres_db = PostgreSQLDatabase()
app_with_postgres = Application(postgres_db)
app_with_postgres.save_data({"name": "Jane"})

              

Inserting {'name': 'John'} into MySQL database
Inserting {'name': 'Jane'} into PostgreSQL database
